<a href="https://colab.research.google.com/github/NeerajHazarika/ENITS_DataMining_WS24/blob/main/Week_8/Assignment_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Credit Card Fraud Detection revised

we return to the credit card use case (last time we used Random Forests).


## Pre-Stage
Use your pre-processing from the last assignment as input.


In [2]:
#download data
#get the data
!git clone https://github.com/keuperj/DATA.git

fatal: destination path 'DATA' already exists and is not an empty directory.


In [3]:
!unzip DATA/creditcard.csv.zip


Archive:  DATA/creditcard.csv.zip
replace creditcard.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [4]:
import pandas as pd
data = pd.read_csv("creditcard.csv")

In [5]:
# prompt: print cols and rows and other stats abt data

import pandas as pd
# Print some info about the data
print(data.info())
print("\nShape of the data:")
print(data.shape)
print("\nDescriptive Statistics:")
print(data.describe())
print("\nFirst 5 rows:")
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
# your preprocessing code
from sklearn.model_selection import train_test_split

X = data.drop(['Class', 'Time', 'V1', 'Amount'], axis=1)
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convert to NumPy arrays
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# Task: Train a MLP-Network
* Train and evaluate the model
* Compare the results to the Random Forrest
* Tune the hyper-parameters (like number of layers, neurons per layer, learning-rate and number of epochs ) to optimize the results

See: https://keras.io/api/

In [9]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, precision_recall_curve, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam


In [11]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Handle class imbalance: SMOTE + NearMiss
smote = SMOTE(random_state=42)
nearmiss = NearMiss()
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Define the MLP model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile with Adam optimizer and Focal Loss
model.compile(optimizer=Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=64,
    validation_split=0.2,
    shuffle=True,
    callbacks=[early_stopping]
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

# Predict probabilities and tune threshold
y_pred_probs = model.predict(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_threshold = thresholds[np.argmax(f1_scores)]
print(f"Optimal Threshold: {optimal_threshold}")

# Use the optimal threshold for predictions
y_pred = (y_pred_probs > optimal_threshold).astype("int32")

# Classification Report and Metrics
print(classification_report(y_test, y_pred))
roc_auc = roc_auc_score(y_test, y_pred_probs)
f1 = f1_score(y_test, y_pred)
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"F1-Score: {f1:.4f}")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.9614 - loss: 0.0964 - val_accuracy: 0.9980 - val_loss: 0.0074
Epoch 2/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 34s 3ms/step - accuracy: 0.9938 - loss: 0.0192 - val_accuracy: 0.9991 - val_loss: 0.0041
Epoch 3/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9961 - loss: 0.0129 - val_accuracy: 0.9993 - val_loss: 0.0030
Epoch 4/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.9972 - loss: 0.0096 - val_accuracy: 0.9994 - val_loss: 0.0029
Epoch 5/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9979 - loss: 0.0079 - val_accuracy: 0.9994 - val_loss: 0.0029
Epoch 6/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9979 - loss: 0.0075 - val_accuracy: 0.9995 - val_loss: 0.0024
Epoch 7/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - accuracy: 0.9982 - loss: 0.0060 - val_accuracy: 0.9996 - val_loss: 0.0019
Epoch 8/50
5687/5687 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - accuracy: 0.9986 - loss: 0

In [13]:
# Generate classification report
report = classification_report(y_test, y_pred, output_dict=True)
mlp_f1 = report['1']['f1-score']

#Compare with random forest
random_forest_f1 = 0.8516129032258064
print(f"\nMLP F1-score: {mlp_f1}")
print(f"Random Forest F1-score: {random_forest_f1}")

if mlp_f1 > random_forest_f1:
    print("MLP model outperforms Random Forest.")
elif mlp_f1 < random_forest_f1:
    print("Random Forest outperforms MLP model.")
else:
    print("MLP model and Random Forest perform equally.")


MLP F1-score: 0.9998417471118848
Random Forest F1-score: 0.8516129032258064
MLP model outperforms Random Forest.
